In [2]:
import pandas as pd
import numpy as np
# from sklearn.preprocessing import PolynomialFeatures
from sklearn import decomposition
import os
import scipy.stats as sc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC


In [3]:
df_co = pd.read_csv('Transformed Data/Transformed_Co', index_col = 0)
df_pt = pd.read_csv('Transformed Data/Transformed_Pt' , index_col = 0)

In [4]:
df_co_len = df_co.shape[0]
df_pt_len = df_pt.shape[0]

In [5]:
df_co_pca = pd.DataFrame(df_co)
df_pt_pca = pd.DataFrame(df_pt)

In [6]:
y1 = pd.Series([0]*df_co_len)
y1.shape

(92,)

In [7]:
y2 = pd.Series([1]*df_pt_len, index = range(df_co_len-1,(df_co_len + df_pt_len)-1))
y2.shape

(214,)

In [8]:
y = pd.concat([y1,y2]) 
y.shape

(306,)

In [9]:
X = pd.concat([df_co_pca, df_pt_pca])

In [15]:
X_train, X_test, y_train1, y_test1 = train_test_split(X, y)
y_train = pd.DataFrame(y_train1)
y_test = pd.DataFrame(y_test1)


print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)


(229, 133)
(77, 133)
(229, 1)
(77, 1)


In [18]:
scaler = MinMaxScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)


In [19]:

    
for this_C in ([0.1, 1, 15, 250, 500, 2000]):

    clf = SVC(kernel = 'linear', C = this_C).fit(X_train, np.ravel(y_train))
    print(' C = {:.2f}, accuracy = {}'.format( this_C, clf.score(X_test, np.ravel(y_test))))
 

 C = 0.10, accuracy = 0.7792207792207793
 C = 1.00, accuracy = 0.8181818181818182
 C = 15.00, accuracy = 0.8571428571428571
 C = 250.00, accuracy = 0.7792207792207793
 C = 500.00, accuracy = 0.8181818181818182
 C = 2000.00, accuracy = 0.8181818181818182


In [20]:
C = 15

clf = SVC(kernel = 'linear', C = C).fit(X_train, np.ravel(y_train))
print(' C = {:.2f}, accuracy = {}'.format( this_C, clf.score(X_test, np.ravel(y_test))))
 

 C = 2000.00, accuracy = 0.8571428571428571
